In [10]:
import json
import os
import sys
from datetime import datetime
from typing import *

from aizynthfinder.aizynthfinder import AiZynthFinder


def disablePrint():
    sys.stdout = open(os.devnull, 'w')


def enablePrint():
    sys.stdout = sys.__stdout__


def generate_time_tag() -> str:
    d = datetime.now()
    return d.strftime("%d%m%y_%H%M")


def setup_finder(config_path: str = "../data/config.yml"):
    finder = AiZynthFinder(config_path)
    finder.stock.select("zinc")
    finder.expansion_policy.select("uspto")
    finder.filter_policy.select("uspto")
    return finder


def load_smiles(smiles_path: str) -> List[str]:
    print(f"Loading SMILES from '{smiles_path}'.")
    with open(smiles_path, mode="r", encoding="utf-8") as f:
        smiles_list = [smiles for smiles in f.read().split("\n")]
    print(f"Loaded {len(smiles_list)} SMILES.")
    return smiles_list


def experiment_with_C(finder: AiZynthFinder, C_values: List[float],
                      smiles_path: str = "../data/smiles_list.txt") -> Dict[float, List[Dict[str, str]]]:
    smiles_list = load_smiles(smiles_path)
    result_dict = {}
    for C in C_values:
        finder.config.C = C
        print(f"Running finder with C set to {finder.config.C}.")
        stats_list = process_smiles_list(finder, target_smiles=smiles_list)
        result_dict[C] = stats_list
    return result_dict


def process_smiles_list(finder: AiZynthFinder, target_smiles: List[str]) -> List[Dict[str, str]]:
    stats_list: List[Dict[str, str]] = []
    for smiles in target_smiles:
        finder.target_smiles = smiles
        finder.tree_search()
        finder.build_routes()
        stats_list.append(finder.extract_statistics())
    return stats_list


def get_smiles_routes(finder: AiZynthFinder, target_smiles: List[str]) -> List[Dict[str, str]]:
    routes = []
    for smiles in target_smiles:
        finder.config.time_limit = 10
        finder.target_smiles = smiles
        finder.tree_search()
        finder.build_routes()
        routes.append(finder.analysis)
    return routes


def main(verbose=False):
    if not verbose:
        disablePrint()

    finder = setup_finder()

    time_tag = generate_time_tag()

    C_values = [1.0, 1.2, 1.4]
    result_dict = experiment_with_C(finder=finder, C_values=C_values)

    with open(f"../data/result_dict_json_{time_tag}.json", mode="w", encoding="utf-8") as json_file:
        json.dump(result_dict, json_file)

In [3]:
main(verbose=True)

Loading SMILES from '../data/smiles_list.txt'.
Loaded 2 SMILES.
Running finder with C set to 1.0
Running finder with C set to 1.2
Running finder with C set to 1.4


In [4]:
with open("../data/result_dict_json_240621_1921.json", mode="r", encoding="utf-8") as f:
    json = json.load(f)

In [5]:
json

{'1.0': [{'target': 'Cc1cccc(C)c1N(CC(=O)Nc1ccc(-c2ncon2)cc1)C(=O)C1CCS(=O)(=O)CC1',
   'search_time': 94.17635321617126,
   'first_solution_time': 12.459070682525635,
   'first_solution_iteration': 10,
   'number_of_nodes': 461,
   'max_transforms': 7,
   'max_children': 16,
   'number_of_routes': 97,
   'number_of_solved_routes': 24,
   'top_score': 0.994039853898894,
   'is_solved': True,
   'number_of_steps': 2,
   'number_of_precursors': 3,
   'number_of_precursors_in_stock': 3,
   'precursors_in_stock': 'O=C(O)C1CCS(=O)(=O)CC1, Cc1cccc(C)c1NCC(=O)O, Nc1ccc(-c2ncon2)cc1',
   'precursors_not_in_stock': '',
   'policy_used_counts': {'uspto': 460}},
  {'target': 'Cc1cccc(C)c1N(CC(=O)Nc1ccc(-c2ncon2)cc1)C(=O)C1CCS(=O)(=O)CC1',
   'search_time': 92.66990780830383,
   'first_solution_time': 10.344624757766724,
   'first_solution_iteration': 10,
   'number_of_nodes': 461,
   'max_transforms': 7,
   'max_children': 16,
   'number_of_routes': 97,
   'number_of_solved_routes': 24,
   'top_s

In [25]:
finder = setup_finder()

In [26]:
def get_smiles_routes(finder: AiZynthFinder, target_smiles: List[str]) -> List[Dict[str, str]]:
    routes = []
    for smiles in target_smiles:
        finder.target_smiles = smiles
        finder.tree_search(show_progress=True)
        finder.build_routes()
        routes.append(finder.analysis.sort_nodes(min_return=2, max_return=3))
    return routes
a = get_smiles_routes(finder=finder, target_smiles=["Cc1cccc(C)c1N(CC(=O)Nc1ccc(-c2ncon2)cc1)C(=O)C1CCS(=O)(=O)CC1"])

In [27]:
print(a)

[([<aizynthfinder.mcts.node.Node object at 0x7f0d2cf39250>, <aizynthfinder.mcts.node.Node object at 0x7f0d2b12e1f0>, <aizynthfinder.mcts.node.Node object at 0x7f0d2a882610>], [0.994039853898894, 0.994039853898894, 0.994039853898894])]
